In [ ]:
#Importing Required Libraries
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json
import collections
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import string
import re
import collections
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import string
import re
import math
import time
import random
from lxml import etree

In [ ]:
#Loading Environment Variables
load_dotenv()

In [ ]:
#Loading Required API Keys
springer_key = os.getenv('springer_api_key')
elsevier_key = os.getenv('elsevier_api_key')

In [ ]:
#Retrieving First Metadata Result from Springer API
n = 1
payload = {'q':'posthuman sort:date openaccess:true', 'p':'100' ,'api_key': springer_key, 's': n}
r = requests.get('http://api.springernature.com/meta/v2/json', params=payload)

In [ ]:
#Processing Response into Data Frame
data_dict = r.json()
df = pd.DataFrame(data_dict["records"])

In [ ]:
#Retrieving All Metadata Results from Springer API
total_num = int(data_dict["result"][0]["total"])
for i in range(math.ceil((total_num-100)/100)):
    n+=100
    payload = {'q':'posthuman sort:date openaccess:true', 'p':'100' ,'api_key': springer_key, 's': n}
    r = requests.get('http://api.springernature.com/meta/v2/json', params=payload)
    data_dict = r.json()
    df = pd.concat([df, pd.DataFrame(data_dict["records"])], axis=0, ignore_index=True)

In [ ]:
#Retrieving Text Results from Springer API
text_list = []
for i in range(len(df)):
    doi = df.loc[i, 'doi']
    doi_q = "doi:" + doi
    querry = {'q': doi_q,'api_key': springer_key}
    r = requests.get('http://api.springernature.com/openaccess/jats', params=querry)
    jats_root = etree.fromstring(r.content)
    body = jats_root.xpath("//body//text()")
    text_content = ' '.join(body)
    text_list.append([doi,text_content])
    time.sleep(8 + random.randint(3, 5))

In [ ]:
#Saving Data Frames as CSV
df.to_csv("data/springer_data_meta.csv", index=False)
pd.DataFrame(text_list).to_csv("data/springer_data_text.csv", index=False)

In [ ]:
#Retrieving First Search Result from Elsevier API
n=0
payload = {'query': 'posthuman', 'start': n, 'count': '100' ,'apiKey': elsevier_key}
r = requests.get('https://api.elsevier.com/content/search/sciencedirect', params=payload)

In [ ]:
#Processing Search Results into Data Frame
data_el = r.json()
data_el_df = pd.DataFrame(data_el['search-results']['entry'])

In [ ]:
#Retrieving All Search Results from Elsevier API
total_n = int(data_el['search-results']['opensearch:totalResults'])
for i in range(math.ceil((total_n-100)/100)):
    n+=100
    payload = {'query': 'posthuman', 'start': n, 'count': '100' ,'apiKey': elsevier_key}
    r = requests.get('https://api.elsevier.com/content/search/sciencedirect', params=payload)
    data_el = r.json()
    data_el_df = pd.concat([data_el_df, pd.DataFrame(data_el["search-results"]['entry'])], axis=0, ignore_index=True)

In [ ]:
#Counting Open Access Articles
data_el_df.value_counts("openaccess")

In [ ]:
#Filtering for Open Access Articles
df_elsevier = data_el_df.loc[data_el_df['openaccess'] == True]
df_elsevier = df_elsevier.reset_index()

In [ ]:
#Retrieving Texts of Open Access Articles from Elsevier API
doi = []
text = []
for i in range(len(df_elsevier)):
    call = "https://api.elsevier.com/content/article/pii/" + df_elsevier["pii"][i]
    header = {'Accept': 'application/json'}
    payload = {'apiKey': elsevier_key}
    r = requests.get(call, params=payload, headers=header)
    data_dict = r.json()
    doi.append(data_dict['full-text-retrieval-response']['coredata']['prism:doi'])
    text.append(data_dict['full-text-retrieval-response']['originalText'])
    time.sleep(8 + random.randint(3, 5))

In [ ]:
#Converting Results into Data Frame
text_elsevier = pd.DataFrame({'doi':doi, 'full-text':text})

In [ ]:
#Retriving First Metadata Results from Elsevier API
call = "https://api.elsevier.com/content/article/pii/" + df_elsevier["pii"][0]
header = {'Accept': 'application/json'}
payload = {'apiKey': elsevier_key}
r = requests.get(call, params=payload, headers=header)

In [ ]:
#Processing Result into Data Frame
data_el = json.loads(r.content)
meta_elsevier = pd.DataFrame.from_dict(data_el['full-text-retrieval-response']['coredata'], orient="index").transpose()

In [ ]:
#Retrieving Metadata for Remaining Elsevier Batch
for i in range(1, len(df_elsevier)):
    call = "https://api.elsevier.com/content/article/pii/" + df_elsevier["pii"][i]
    header = {'Accept': 'application/json'}
    payload = {'apiKey': elsevier_key}
    r = requests.get(call, params=payload, headers=header)
    data_el = json.loads(r.content)
    temp_df = pd.DataFrame.from_dict(data_el['full-text-retrieval-response']['coredata'], orient="index").transpose()
    meta_elsevier = pd.concat([meta_elsevier, temp_df], axis=0, ignore_index=True)
    time.sleep(5 + random.randint(3, 5))

In [ ]:
#Saving Elsevier Data into CSV Files
text_elsevier.to_csv("data/elsevier_data_text.csv", index=False)
meta_elsevier.to_csv("data/elsevier_data_meta.csv", index=False)